In [1]:
# # Do not forget to check if your notebook log collection plugin is working
from mining_extension import check_logging
check_logging("http://3.249.245.244:9999")

Request successful!


In [ ]:
!pip install pandas

In [2]:
import numpy as np

In [3]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [4]:
train_features

,filename,cell_type,cell_number,execution_count,linesofcomment,linesofcode,variable_count,function_count,text/plain,image/png,...,helper_functions,load_data,data_exploration,data_preprocessing,evaluation,modelling,prediction,result_visualization,save_results,comment_only
0,nb_54880.ipynb,code,0,1,0.0,2.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,nb_54880.ipynb,code,1,2,1.0,4.0,4.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,nb_54880.ipynb,code,2,3,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,nb_54880.ipynb,code,3,4,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,nb_54880.ipynb,code,4,5,0.0,1.0,1.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,nb_95821.ipynb,code,27,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
5829,nb_95821.ipynb,code,29,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
5830,nb_95821.ipynb,code,31,-1,0.0,3.0,2.0,0.0,0,0,...,0,0,1,1,0,0,0,0,0,0
5831,nb_95821.ipynb,code,33,-1,5.0,17.0,14.0,0.0,0,0,...,0,0,0,0,1,1,1,0,0,0


In [5]:
test_features.columns 

Index(['filename', 'cell_type', 'cell_number', 'execution_count',
       'linesofcomment', 'linesofcode', 'variable_count', 'function_count',
       'text/plain', 'image/png', 'text/html', 'execute_result',
       'display_data', 'stream', 'error', 'text', 'comment',
       'code_line_before', 'code_line_after', 'markdown_heading',
       'packages_info'],
      dtype='object')

In [6]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numerical_cols = test_features.select_dtypes(include=numerics).columns
numerical_cols

Index(['cell_number', 'execution_count', 'linesofcomment', 'linesofcode',
       'variable_count', 'function_count', 'text/plain', 'image/png',
       'text/html', 'execute_result', 'display_data', 'stream', 'error'],
      dtype='object')

In [7]:
train_num_features = train_features[numerical_cols]
val_num_features = validation_features[numerical_cols] 
test_num_features = test_features[numerical_cols]

In [8]:
def get_mark_down(df):
    mark_down = df['markdown_heading'].to_numpy()
    arr = []

    for a in mark_down:
        if a == 0:
            arr.append('')
        else:
            arr.append(a[0])

    return arr       

In [9]:
mark_down_train = get_mark_down(train_features)
mark_down_val = get_mark_down(validation_features)
mark_down_test = get_mark_down(test_features)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(mark_down_train)
X_md_train = vectorizer.transform(mark_down_train)
X_md_val = vectorizer.transform(mark_down_val)
X_md_test = vectorizer.transform(mark_down_test)
X_md_train.shape, X_md_val.shape, X_md_test.shape

((5833, 3145), (1927, 3145), (1918, 3145))

In [11]:
from scipy.sparse import hstack

In [12]:
X_train = hstack((train_num_features.to_numpy(), X_md_train))
X_val = hstack((val_num_features.to_numpy(), X_md_val))
X_test = hstack((test_num_features.to_numpy(), X_md_test))

In [13]:
!pip install scikit-learn

In [14]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_features['primary_label'])
y_train = le.transform(train_features['primary_label'])
y_val = le.transform(validation_features['primary_label'])

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score

best_score = 0
best_depth = 0

for i in range(40, 71, 5):

    clf = RandomForestClassifier(max_depth=i, n_estimators=100)

    clf.fit(X_train, y_train)

    pred = clf.predict(X_train)
    print(i, " train", f1_score(pred, y_train, average='weighted'), end = '     ')

    pred = clf.predict(X_val)
    score = f1_score(pred, y_val, average='weighted')
    print("val", score)
    
    if score > best_score:
        best_score = score
        best_depth = i

print("Best score", best_score, "with max_depth", best_depth)

40  train 0.8289543097328497     val 0.6272274053269697
45  train 0.8492641798393837     val 0.63974764177321
50  train 0.8693707970393552     val 0.6268457829190593
55  train 0.8849093168541199     val 0.6368663493137434
60  train 0.9000485698822421     val 0.6328690270445206
65  train 0.9165271664929046     val 0.6356656043440099
70  train 0.9350751682630112     val 0.6221354250863554
Best score 0.63974764177321 with max_depth 45


In [21]:
clf = RandomForestClassifier(max_depth=best_depth, n_estimators=500)

X_train_val = np.concatenate((X_train.toarray(), X_val.toarray()), axis=0)
y_train_val = np.concatenate((y_train, y_val), axis=0)

clf.fit(X_train_val, y_train_val)

pred = clf.predict(X_train_val)
print(i, " train", f1_score(pred, y_train_val, average='weighted'), end = '     ')

pred = clf.predict(X_test)

70  train 0.8432318148764175     

In [22]:
pred.shape

(1918,)

In [23]:
vals = le.inverse_transform(pred)
vals

array(['helper_functions', 'data_preprocessing', 'data_preprocessing',
       ..., 'helper_functions', 'data_preprocessing', 'helper_functions'],
      dtype=object)

In [26]:
df = pd.DataFrame({"primary_label" : vals })
df.head()

,primary_label
0,helper_functions
1,data_preprocessing
2,data_preprocessing
3,data_preprocessing
4,data_exploration


In [28]:
df.to_csv('submission.csv', index=True)